## PFNが開発したチューニング手法
### GridSearchより早く、高いスコアが期待できる
目的関数を自分で作成する必要がある。目的関数の最小化のみに対応しているので、正解率を上げたい場合は1から引くなどする必要がある、　　
内部のアルゴリズムはベイズ最適化を用いている。一度まとめて調べる必要があるな。

## データセット準備
自分で作成する目的関数で、訓練データではなく、検証データの値から求めた値を最小化したいため、検証データを作成する

In [6]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_boston
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

boston = load_boston()
X, y = boston["data"], boston["target"]
# 訓練、テスト分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=114514)
n_train = int(X_train.shape[0] * 0.75)
X_train, X_val = X_train[:n_train], X_train[n_train:]
y_train, y_val = y_train[:n_train], y_train[n_train:]

# 標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## 最小化したい目的関数を作成
optunaで最小化する値。回帰ならMSE,分類問題なら正解率の1との差をとった値を返すようにする。  
パラメータ範囲の指定があり、モデルのインスタンスを作成して、学習を内部で行う。  
評価指標は検証用データの値になるようにする

In [7]:
import optuna
from sklearn.metrics import mean_squared_error

def objective(trial):
    # パラメータの作成
    svr_c = trial.suggest_loguniform('svc_c', 1e0,1e2)
    epsilon = trial.suggest_loguniform('epsilon', 1e-1,1e1)
    # モデルの作成
    svm_reg=SVR(C=svr_c,epsilon=epsilon)
    # 訓練データで学習
    svm_reg.fit(X_train,y_train)
    # 検証データの予測
    y_pred=svm_reg.predict(X_val)
    # 最小化したい評価指標。検証データでの値を用いる
    return mean_squared_error(y_val,y_pred)

In [8]:
study=optuna.create_study()
study.optimize(objective,n_trials=100)

[I 2019-07-21 22:35:05,819] Finished trial#0 resulted in value: 9.605281851865122. Current best value is 9.605281851865122 with parameters: {'svc_c': 4.777670748856879, 'epsilon': 1.917825768685567}.
[I 2019-07-21 22:35:05,905] Finished trial#1 resulted in value: 20.491311224302148. Current best value is 9.605281851865122 with parameters: {'svc_c': 4.777670748856879, 'epsilon': 1.917825768685567}.
[I 2019-07-21 22:35:05,996] Finished trial#2 resulted in value: 8.341645250742767. Current best value is 8.341645250742767 with parameters: {'svc_c': 39.61913822097133, 'epsilon': 0.5201896153491742}.
[I 2019-07-21 22:35:06,087] Finished trial#3 resulted in value: 9.257065148611273. Current best value is 8.341645250742767 with parameters: {'svc_c': 39.61913822097133, 'epsilon': 0.5201896153491742}.
[I 2019-07-21 22:35:06,173] Finished trial#4 resulted in value: 12.27971187379273. Current best value is 8.341645250742767 with parameters: {'svc_c': 39.61913822097133, 'epsilon': 0.520189615349174

[I 2019-07-21 22:35:13,548] Finished trial#80 resulted in value: 7.654688262604661. Current best value is 7.6338360638145355 with parameters: {'svc_c': 13.154969759520327, 'epsilon': 0.9485257989214145}.
[I 2019-07-21 22:35:13,646] Finished trial#81 resulted in value: 9.695266129492785. Current best value is 7.6338360638145355 with parameters: {'svc_c': 13.154969759520327, 'epsilon': 0.9485257989214145}.
[I 2019-07-21 22:35:13,748] Finished trial#82 resulted in value: 7.620054392428591. Current best value is 7.620054392428591 with parameters: {'svc_c': 21.877652892973153, 'epsilon': 1.3686214992597119}.
[I 2019-07-21 22:35:13,848] Finished trial#83 resulted in value: 8.023000889084235. Current best value is 7.620054392428591 with parameters: {'svc_c': 21.877652892973153, 'epsilon': 1.3686214992597119}.
[I 2019-07-21 22:35:13,948] Finished trial#84 resulted in value: 9.461341650532345. Current best value is 7.620054392428591 with parameters: {'svc_c': 21.877652892973153, 'epsilon': 1.36

## 最適なハイパーパラメータを確認

In [9]:
print(study.best_params)
print(study.best_value)
print(study.best_trial)

{'svc_c': 13.544150068871302, 'epsilon': 0.8739308486904945}
7.614515425185181
FrozenTrial(number=93, state=<TrialState.COMPLETE: 1>, value=7.614515425185181, datetime_start=datetime.datetime(2019, 7, 21, 22, 35, 14, 808806), datetime_complete=datetime.datetime(2019, 7, 21, 22, 35, 14, 906583), params={'svc_c': 13.544150068871302, 'epsilon': 0.8739308486904945}, distributions={'svc_c': LogUniformDistribution(low=1.0, high=100.0), 'epsilon': LogUniformDistribution(low=0.1, high=10.0)}, user_attrs={}, system_attrs={'_number': 93}, intermediate_values={}, params_in_internal_repr={'svc_c': 13.544150068871302, 'epsilon': 0.8739308486904945}, trial_id=93)


## 得られたハイパーパラメータで学習

In [12]:
from sklearn.metrics import r2_score
model=SVR(C=13.544150068871302,epsilon=0.8739308486904945)
model.fit(X_train,y_train)
y_pred_val=model.predict(X_val)
y_pred_test=model.predict(X_test)

print('R^2 validation:',r2_score(y_val,y_pred_val))
print('R^2 test:',r2_score(y_test,y_pred_test))

R^2 validation: 0.8666883650210149
R^2 test: 0.9147160238229803


## XGBoostの場合

In [13]:
import optuna
import xgboost as xgb
# Objective Functionの作成
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 0, 1000)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    subsample = trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1)
    colsample_bytree = trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1)
    xgboost_reg = xgb.XGBRegressor(
        random_state=42,
        n_estimators = n_estimators,
        max_depth = max_depth,
        min_child_weight = min_child_weight,
        subsample = subsample,
        colsample_bytree = colsample_bytree,
    )
        # 訓練データで学習
    xgboost_reg.fit(X_train,y_train)
    # 検証データの予測
    y_pred=xgboost_reg.predict(X_val)
    # 最小化したい評価指標。検証データでの値を用いる
    return mean_squared_error(y_val,y_pred)

In [15]:
study=optuna.create_study()
study.optimize(objective,n_trials=100)

print(study.best_params)
print(study.best_value)
print(study.best_trial)

[22:44:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2019-07-21 22:44:15,899] Finished trial#0 resulted in value: 8.682003547785644. Current best value is 8.682003547785644 with parameters: {'n_estimators': 788, 'max_depth': 8, 'min_child_weight': 10, 'subsample': 0.5, 'colsample_bytree': 0.7}.


[22:44:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2019-07-21 22:44:16,200] Finished trial#1 resulted in value: 9.205659384699254. Current best value is 8.682003547785644 with parameters: {'n_estimators': 788, 'max_depth': 8, 'min_child_weight': 10, 'subsample': 0.5, 'colsample_bytree': 0.7}.


[22:44:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[22:45:03] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2019-07-21 22:45:04,487] Finished trial#98 resulted in value: 8.731387202942802. Current best value is 6.698855612684564 with parameters: {'n_estimators': 850, 'max_depth': 14, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.7}.


[22:45:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2019-07-21 22:45:05,069] Finished trial#99 resulted in value: 8.330766218992352. Current best value is 6.698855612684564 with parameters: {'n_estimators': 850, 'max_depth': 14, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.7}.


{'n_estimators': 850, 'max_depth': 14, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.7}
6.698855612684564
FrozenTrial(number=92, state=<TrialState.COMPLETE: 1>, value=6.698855612684564, datetime_start=datetime.datetime(2019, 7, 21, 22, 44, 59, 262417), datetime_complete=datetime.datetime(2019, 7, 21, 22, 45, 0, 27371), params={'n_estimators': 850, 'max_depth': 14, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.7}, distributions={'n_estimators': IntUniformDistribution(low=0, high=1000), 'max_depth': IntUniformDistribution(low=1, high=20), 'min_child_weight': IntUniformDistribution(low=1, high=20), 'subsample': DiscreteUniformDistribution(low=0.5, high=0.9, q=0.1), 'colsample_bytree': DiscreteUniformDistribution(low=0.5, high=0.9, q=0.1)}, user_attrs={}, system_attrs={'_number': 92}, intermediate_values={}, params_in_internal_repr={'n_estimators': 850.0, 'max_depth': 14.0, 'min_child_weight': 3.0, 'subsample': 0.7, 'colsample_bytree': 0.7}, trial_id=92)


In [18]:
from sklearn.metrics import r2_score
model=xgb.XGBRegressor(n_estimators= 850,
                        max_depth= 14,
                       min_child_weight= 3,
                       subsample= 0.7,
                       colsample_bytree= 0.7)
model.fit(X_train,y_train)
y_pred_val=model.predict(X_val)
y_pred_test=model.predict(X_test)

print('R^2 validation:',r2_score(y_val,y_pred_val))
print('R^2 test:',r2_score(y_test,y_pred_test))

[23:03:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
R^2 validation: 0.8718783141603175
R^2 test: 0.8706309172641004
